!!! VERY IMPORTANT INFORMATION AFTER RELEASE OF TENSORFLOW 2.8.0 !!!

It seems that colab uses an old version of cuDNN on board. This is a problem
because all of our code (with packages) are "compiled" on another version.
Because of this

To still be able to run everything on this notebook, please do the following steps:

* Register an Nvidia Developer account (https://developer.nvidia.com/)
* Go to https://developer.nvidia.com/rdp/cudnn-archive
* Go to "Download cuDNN v8.1.0 (January 26th, 2021), for CUDA 11.0,11.1 and 11.2"
* Find and download "cuDNN Runtime Library for Ubuntu18.04 x86_64 (Deb)"
* Put the downloaded file to the root directory of this colab machine

In [ ]:
#uncomment this and run this if necessary
# !dpkg -i "libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb"
# !pip install OpenNMT-tf

As of 01.02.2022 everything should work fine after this

# 0 Information

## 0.1 Prerequisites

### 0.1.1 Git
* Should be installed
* Should be available via path, so that it is possible to call `git clone ...` directly from the console

### 0.1.2 `tar` tool
TODO

### 0.1.3 Python
It is recommended to use version 3.8+. The oldest version on which
this notebook and all codebase scripts have been tested is 3.7.

## 0.2 Recommendations

### 0.2.1 Configuration
The current setup of this file allow you to completely run all the modules of the project without any problems.
If you wish to simply execute all the functions, and you don't require special configuration or output directories
then please stick to the configuration that is provided here by default.

### 0.2.2 Virtual environment
It is recommended to run this notebook in a separate python virtual environment.
`conda`, `venv` or any other virtual environment creation options are suitable for this.
Even though this is not a requirement to run this notebook it is very
recommended using a virtual environment

Here is a small tutorial on how to set a virtual environment with `venv`:

TODO

# 1 Setup

## 1.1 Config

### 1.1.1 `CLONE_GITHUB_REPO`
Set to true if you only have the notebook
and you don't have the other parts of codebase.
This option will allow the notebook to pull
all the codebase form the github repo of this project

In [ ]:
CLONE_GITHUB_REPO = True

### 1.1.2 `MAIN_PACKAGE_PATH`
This is the path to the codebase directory, where all the subpackages and modules are stored.
If `CLONE_GITHUB_REPO` is `True` then this will be interpreted as the path to which the GitHub
Repo will be cloned. This path can be provided as an absolute path or as a relative path
to the file of this notebook.

*   If you run this notebook "out of the box" i.e. with the GitHub repo already cloned,
    please specify `MAIN_PACKAGE_PATH = '.'`
    *   When is this applicable: this file's path is `repo_root/main.ipynb` where `repo_root`
        is the path to the cloned GitHub repo of this project with the whole codebase
*   If this notebook is not in the context of the cloned GitHub repo of this project then
    please specify your custom path. IMPORTANT: the final directory, in which the repo will be cloned
    **should always have the name`studienprojekt_cv_rvv`**
    * Allowed paths (unix-like, windows are analogous):
        * `/home/user/Desktop/studienprojekt_cv_rvv`
        * `./studienprojekt_cv_rvv`
        * `./another/example/directory/studienprojekt_cv_rvv`
        * `.` on condition that the current working directory has name `studienprojekt_cv_rvv`
        * `..` on condition that the parent directory of the current working directory is `studienprojekt_cv_rvv`
        * `../../just/another/example/studienprojekt_cv_rvv`
    * Disallowed paths (unix-like, windows are analogous):
        * `/home/user/Desktop/studienprojekt`
        * `./project`
        * `./another/example/directory`
        * `.` on condition that the current working directory **does not have** name `studienprojekt_cv_rvv`
        * `..` on condition that the parent directory of the current working directory **is not** `studienprojekt_cv_rvv`
        * `../../just/another/example`

In [1]:
MAIN_PACKAGE_PATH = './studienprojekt_cv_rvv' if CLONE_GITHUB_REPO else '.'

### 1.1.3 `SCRIPTS_EXEC_MODE`
It is possible to execute all scripts in `verbose` mode to display
all logs that are displayed by the scripts themselves
and also by all the subprocesses that they start. However, if you use `silent` mode
then no logs will be displayed. Only important messages like errors and
important execution checkpoints will be logged in silent mode.

In [ ]:
SCRIPTS_EXEC_MODE = 'verbose'

### 1.1.4 `EVAL_SPLIT_RATIO`
Specify a float value from `0.0` to `1.0` which defines how big the part of images should
be used for evaluation. Value of `0.2` means that 20% of all the images in the provided dataset
should be used for evaluation. The rest is used for training

In [ ]:
EVAL_SPLIT_RATIO = 0.2

### 1.1.5 `NUM_TRAIN_STEPS`
The number of training steps during the model training process. A training step is ...TODO...

In [ ]:
NUM_TRAIN_STEPS = 1000

## 1.2 Definitions of constants
*Note: You still have to run these cells for further cells to function correctly.
But please only edit this part of the notebook if you know what you are doing*

In [1]:
GITHUB_REPO_LINK = 'https://github.com/Maks0bs/study-project-cv-real-vs-virtual.git'
CHECKPOINT_EVERY_N = 100

## 1.3 Imports
Here all external packages are imported

In [ ]:
import shutil
import os


# 2 Workspace preparation
All of these cells should be run only for one time when we "start from scratch" with this notebook

## 2.1 Codebase
Pull the codebase from GitHub if necessary and verify its installation

In [ ]:
if CLONE_GITHUB_REPO:
    !git clone {GITHUB_REPO_LINK} {MAIN_PACKAGE_PATH}

# TODO: verify codebase installation

## 2.2 Package setup
Install the project package to the python (virtual) environment

In [ ]:
main_package_parent = os.path.dirname(MAIN_PACKAGE_PATH)
shutil.copy(
    os.path.join(MAIN_PACKAGE_PATH, 'setup.py'),
    main_package_parent
)
!pip install -e {main_package_parent}

# 3 TFOD API and pretrained model
Here we install Tensorflow Object Detection API and
load the pretrained model that is specified in the config.
Generally, the TFOD API should only be installed once.
However, several pretrained models can be loaded (when the config is changed)
and in this case it is required to run these cells once again
for the new pretrained model to be used during training.

In [ ]:
!python {os.path.join(MAIN_PACKAGE_PATH, 'setup', 'workspace.py')} -m {SCRIPTS_EXEC_MODE} -v "both"

## 4 Dataset preparation
Here we create the processed dataset with splitting into train and test sets.
Please put your raw perception data in ./studienprojekt_cv_rvv/data/datasets/raw/my_dataset

In [ ]:
!python {os.path.join(MAIN_PACKAGE_PATH, 'data', 'preparation.py')} -m {SCRIPTS_EXEC_MODE} -sr {EVAL_SPLIT_RATIO}

## 5 Train the model
Train the model on the created TF records

In [ ]:
!python {os.path.join(MAIN_PACKAGE_PATH, 'train', 'train.py')} -m {SCRIPTS_EXEC_MODE} --num_train_steps {NUM_TRAIN_STEPS} --checkpoint_every_n {CHECKPOINT_EVERY_N}

## 6 Evaluate the model

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format('/content/studienprojekt_cv_rvv/tfod_api/object_detection/model_main_tf2.py', '/content/studienprojekt_cv_rvv/models/trained/my_model' ,'/content/studienprojekt_cv_rvv/models/trained/my_model/pipeline.config', '/content/studienprojekt_cv_rvv/models/trained/my_model')

In [ ]:
print(command)

In [ ]:
!{command}

## 7 Image Object Detection

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
# Image Object Detection

PIPELINE_CONFIG_FILE = ''
LABELMAP_FILE = ''
NEW_CKPT_PATH = ''

configs = config_util.get_configs_from_pipeline_file(PIPELINE_CONFIG_FILE)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(NEW_CKPT_PATH, 'ckpt-XXX')).expect_partial()


@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(LABELMAP_FILE)
IMAGE_PATH = ''

In [ ]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=10,
            min_score_thresh=.25,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.savefig('test.png')
plt.show()